In [1]:
%matplotlib inline
import os
import ipywidgets as widgets
from matplotlib import pyplot as plt

# size of latent z vector
nz = widgets.IntText(
    value=512,
    description='nz'
)

ngf =  widgets.IntText(
    value=32,
    description='ngf'
)

ndf =  widgets.IntText(
    value=16,
    description='ndf'
)

n_epoch = widgets.IntText(
    value=25,
    description='n epoch'
)

lr = widgets.FloatText(
    value = 0.002,
    description = 'learning rate'
)

beta1 = widgets.FloatText(
    value = 0.5,
    description = 'beta 1'
)

im_size = widgets.IntText(
    value = 64,
    description = 'im size'
)

file_path = widgets.Text(
    description = 'path'
)

u_box = widgets.HBox([nz, ngf, ndf, n_epoch])
b_box = widgets.HBox([lr, beta1,im_size])
v_b = widgets.VBox([u_box,b_box,file_path])

display(v_b)

In [2]:
import h5py
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch

BATCH_SIZE = 5

class ctimage(Dataset):
    def __init__(self, path):
        self.img = h5py.File(path, 'r')['data']
    def __len__(self):
        return self.img.shape[0]
    def __getitem__(self,idx):
        return torch.from_numpy(np.expand_dims(self.img[idx],axis = 0)), 0.9

training_data = ctimage(file_path.value)

dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [3]:
import dcgan
nc = 1

netG = dcgan.make_generator_model(im_size.value, nz.value, nc, ngf.value)
netD = dcgan.make_discriminator_model(im_size.value, nc, ndf.value)


In [4]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.BCELoss()

device = "cuda:0" if torch.cuda.is_available() else "cpu"

print(device)

if(torch.cuda.is_available()):
    netG = netG.cuda()
    netD = netD.cuda()
    criterion = criterion.cuda()

d_optimizer = optim.Adam(netD.parameters(), lr = lr.value, betas = (beta1.value, 0.999))
g_optimizer = optim.Adam(netG.parameters(), lr = lr.value, betas = (beta1.value, 0.999))
    

cuda:0


In [5]:
import time
from torch import tensor
import os.path

gen_iterations = 0
for epoch in range(n_epoch.value):
    start = time.time()
    for i_batch, sample_batched in enumerate(dataloader):
        netD.zero_grad()
        
        real_img = sample_batched[0].to(device, dtype=torch.float)
        real_label = sample_batched[1].to(device, dtype=torch.float)        
        
        #train with real
        output = netD(real_img)
        errD_real = criterion(output, real_label)
        errD_real.backward()
        D_x = output.data.mean()
        
        noise = torch.rand((BATCH_SIZE,nz.value,1,1,1)).to(device, dtype=torch.float)
        fake = netG(noise).detach()
        
        output = netD(fake)
        
        fake_label = tensor([0.0]*BATCH_SIZE, dtype = torch.float).to(device)
        errD_fake = criterion(output, fake_label)
        errD_fake.backward()
        
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        d_optimizer.step()
        
        g_iter = 1
        
        while g_iter != 0:
            netG.zero_grad()
            real_label = tensor([1.0]*BATCH_SIZE, dtype = torch.float).to(device) # fake labels are real for generator cost
            noise = torch.rand((BATCH_SIZE,nz.value,1,1,1)).to(device, dtype=torch.float)
            
            fake = netG(noise)
            output = netD(fake)
            errG = criterion(output, real_label)
            errG.backward()
            D_G_z2 = output.data.mean()
            g_optimizer.step()
            g_iter -= 1
        
        gen_iterations += 1
    print("epoch {} time: {}".format(epoch,time.time()-start))
    work_dir = "training_checkpoints"
    if epoch % 5 == 0:
        torch.save(netG.state_dict(), os.path.join(".",work_dir,"netG_epoch_{}.pth".format(epoch)))
        torch.save(netD.state_dict(), os.path.join(".",work_dir,"netD_epoch_{}.pth".format(epoch)))
        

/home/msam44/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1, 1, 1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/home/msam44/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1, 1, 1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


epoch 0 time: 190.25475215911865


Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

KeyboardInterrupt: 